In [389]:
import pymongo
import datetime
import time
import random 
import requests
from bs4 import BeautifulSoup 
import os
import pickle
import pandas as pd
import numpy as np
import math
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import string
import pprint as pp

user = 'jefferyholden1@outlook.com'
password = 'LinkedInBot12#'

In [521]:
# initialize mongo 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.test_2020_04_25
# db = client.test
links_collection = db.links
jobs_collection = db.jobs

# Login to LinkedIn
---
### To-Do's:
    - [ ] Refactor xpath syntax to //*[@class="btn__primary--large from__button--floating"]

In [214]:
# pages = []
# offset = 25
# # opts = Options()
# # opts.add_argument(f"user-agent={agent}")
driver = webdriver.Chrome('../driver/chromedriver')
driver.get('http://linkedin.com')
try:
    green_bar = driver.find_element_by_class_name('special-report-banner__content')
except :
    pass
if green_bar:
    login = driver.find_element_by_xpath('/html/body/nav/a[3]')
    login.click()
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div/main/div[2]/form/div[1]/input').send_keys(user)
    driver.find_element_by_xpath('/html/body/div/main/div[2]/form/div[2]/input').send_keys(password)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@class="btn__primary--large from__button--floating"]').click()
else:
    driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input').send_keys(user)
    driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input').send_keys(password)   
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button').click()
    driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
# time.sleep(random.randrange(3,8))

# Scrape #1 - Get the Job Links
---
### To-Do's:
    - [ ] Make sure bot gets all results (only got 525/647 results)


In [144]:
# This is how the user should interact with this app eventually
# search_query = str(input('Enter a search query here: ')).lower().replace(' ','%20')
# print(search_query)
# location = str(input('Please enter a search location: ')).lower().replace(' ','%20')
# print(location)

In [215]:
# linkedin search query
location = 'Los%20Angeles'
search_query = 'Data%20Scientist'

# Set pagination for linkedin pages
offset = 0

# Give each link a unique id
scrape_1_id = 1

# Get the first page and extract the number of job results 
driver.get(f'https://www.linkedin.com/jobs/search/?geoId=90000049&keywords={search_query}&location={location}&sortBy=R&start={offset}')
num_results = int(driver.find_element_by_xpath('//*[@class="display-flex t-12 t-black--light t-normal"]').text.split(' ')[0])
num_of_pages = math.ceil(num_results / 25) if num_results < 1000 else 40

# Scrape each linkedin job search page and return the job links
for index in range(num_of_pages):
   
    driver.get(f'https://www.linkedin.com/jobs/search/?geoId=90000049&keywords={search_query}&location={location}&sortBy=R&start={offset}')
    time.sleep(2)  # allow each page to load

    # Load entire page by tabbing 250 times 
    action = ActionChains(driver)
    action.send_keys(Keys.TAB * 500)
    action.perform()

    # finds all the job links and returns a list of links
    container = driver.find_elements_by_xpath('//*[@class="job-card-container__link job-card-list__title disabled ember-view"]')
    
    # iterates throught the link list to extract job links
    for link in container:
        links_collection.insert_one({'scrape_1_id':scrape_1_id,"link":link.get_attribute('href')})
        scrape_1_id += 1
    
    # increment offset to go to next page of results
    offset += 25

# Scape #2 - Extract data from job links

In [ ]:
# scrape all html from job posting
link_list = list(links_collection.find())
raw_html = []

# for idx, link in enumerate(link_list):
#     driver.get(link)
# #   grab the raw html
#     raw_html.append({'job_id':idx,'job':driver.get_source()})
# #     grab the elements off the page

# # turn elements into a json object

In [526]:
links_list = list(links_collection.find())

In [528]:
for index, link in enumerate(links_list):
    driver.get(link['link'])
    time.sleep(5)
    driver.find_element_by_xpath('//*[@class="t-14 t-black--light t-bold artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--tertiary ember-view"]').click()
    # reset variables after every scrape
    job_title = None
    employer_name = None
    employer_location = None
    posted_time = None
    num_of_applicants = None
    job_description = None
    experience_level = None
    employment_type = None
    industry = None
    job_functions = None

    try:        
        # Job Title
        job_title = driver.find_element_by_xpath('//*[@class="jobs-top-card__job-title t-24"]').text
    except:
        pass

    try: 
        # Employer Name
        employer_name = driver.find_element_by_xpath('//*[@class="jobs-top-card__company-url ember-view"]').text
    except:
        pass

    try: 
        # Employer Location
        employer_location = driver.find_element_by_xpath('//*[@class="jobs-top-card__bullet"]').text
    except:
        pass

    try: 
        # Posted Time
        posted_time = driver.find_element_by_xpath('//*[@class="mt1 full-width flex-grow-1 t-14 t-black--light"]').text.split('\n')[1]
    except:
        pass

    try: 
        # Number of Applicants
        num_of_applicants = int(driver.find_element_by_xpath('//*[@class="mt1 full-width flex-grow-1 t-14 t-black--light"]').text.split('\n')[3].split(' ')[0])
    except:
        pass

    try: 
        # Job Description
        job_description = driver.find_element_by_xpath('//*[@class="jobs-box__html-content jobs-description-content__text t-14 t-black--light t-normal"]').text.lower() \
            .translate({ord(ch):' ' for ch in '. \n , - / \ [ ]( )'}).replace('  ', ' ')
    except:
        pass

    try: 
        # Experience Level
        experience_level = driver.find_elements_by_xpath('//*[@class="jobs-box__body js-formatted-exp-body"]')[0].text
    except:
        pass

    try: 
        # Industry 
        industry1 = driver.find_elements_by_xpath('//*[@class="jobs-box__list jobs-description-details__list js-formatted-industries-list"]')
        industry2 = industry1[0].find_elements_by_tag_name('li')
        industry = [ind.text for ind in industry2]
    except:
        pass

    try: 
        # Employment Type
        employment_type = driver.find_elements_by_xpath('//*[@class="jobs-box__body js-formatted-employment-status-body"]')[0].text
    except:
        pass

    try: 
        # Job Functions
        job_functions1 = driver.find_elements_by_xpath('//*[@class="jobs-box__list jobs-description-details__list js-formatted-job-functions-list"]')
        job_functions2 = job_functions1[0].find_elements_by_tag_name('li')
        job_functions = [jf.text for jf in job_functions2]

    except:
        pass

    jobs_collection.insert_one({
        'scrape_time': datetime.datetime.now(),
        'job_id': index,
        'job_title':job_title,
        'employer_name': employer_name,
        'employer_location':employer_location,
        'posted_time':posted_time,
        'num_of_applicants':num_of_applicants,
        'job_description': job_description,
        'experience_level': experience_level,
        'industry': industry,
        'employment_type': employment_type,
        'job_functions':job_functions,
        'url_link' : link['link']
    })
# dictt

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="t-14 t-black--light t-bold artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--tertiary ember-view"]"}
  (Session info: chrome=81.0.4044.122)


In [518]:
driver.get('https://www.linkedin.com/jobs/view/1788219535/?eBP=NotAvailableFromVoyagerAPI&refId=3b8372d8-48cc-416b-837a-191e9a884e48&trk=flagship3_search_srp_jobs')
driver.find_element_by_xpath('//*[@class="t-14 t-black--light t-bold artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--tertiary ember-view"]').click()

In [519]:
# Employment Type
employment_type = driver.find_elements_by_xpath('//*[@class="jobs-box__body js-formatted-employment-status-body"]')[0].text
employment_type

# # Job Functions
job_functions1 = driver.find_elements_by_xpath('//*[@class="jobs-box__list jobs-description-details__list js-formatted-job-functions-list"]')
job_functions2 = job_functions1[0].find_elements_by_tag_name('li')
job_functions = [jf.text for jf in job_functions2]
job_functions

['Engineering', 'Information Technology']